 **install textblob**

In [4]:
!pip3 install textblob
!pip3 install googletrans
!pip3 install pygal
!pip3 install pandas
!pip3 install geopandas
!pip3 install matplotlib
!pip3 install numpy
!pip3 install plotly_express
!pip install google-cloud-translate==2.0.1



Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [17]:
! export GOOGLE_APPLICATION_CREDENTIALS=$(pwd)/key.json
! echo $GOOGLE_APPLICATION_CREDENTIALS

/home/rudda/workspace/data_science/analise_sentimentos/analise sentimentos-1ea51c23cc36.json


**imports**

In [1]:
# -*- coding: utf-8 -*-
from textblob import TextBlob
from IPython.display import SVG, HTML, display
from google.cloud import translate_v2 as translate
import pygal
import datetime
import calendar
import json
import time
import re
import statistics
import numpy



base_html = """
<!DOCTYPE html>
<html>
  <head>
  <script type="text/javascript" src="http://kozea.github.com/pygal.js/javascripts/svg.jquery.js"></script>
  <script type="text/javascript" src="https://kozea.github.io/pygal.js/2.0.x/pygal-tooltips.min.js""></script>
  </head>
  <body>
    <figure>
      {rendered_chart}
    </figure>
  </body>
</html>
"""

<font size="4">**Translate file and salve in other file**</font>

In [18]:
# it need a json Key available on google API console and then RUN
# export GOOGLE_APPLICATION_CREDENTIALS="/home/user/Downloads/my-key.json"

def googleTranslator(text, target):
    target = "en"
    translate_client = translate.Client()
    result = translate_client.translate(text, target_language=target)
    return result


def load():

    dataset = open('data/dataset-v2.txt', 'r', encoding='utf-8')
    data_enUS = open('data/dataset-us-v4.txt', 'a', encoding='utf-8')
    data_error = open('data/dataset-error.json', 'a', encoding='utf-8')

    i = 0; # create an index
    target = "en-US" # target that will be translated
    #emoji pattern
    emoji_pattern = re.compile("["
            u"\U0001F600-\U0001F64F"  # emoticons
            u"\U0001F300-\U0001F5FF"  # symbols & pictographs
            u"\U0001F680-\U0001F6FF"  # transport & map symbols
            u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
            "]+", flags=re.UNICODE)
    aux= []
    canITranslate = False # manual fix
    latestId = '301420767' # we use only manual fix

    for line in dataset:
        try:
            #increment index - just for fun :)
            i  = i + 1
            # parse to json format
            data_json = json.loads(str(line))
            aux = data_json
            #remove emojis from review
            review_str = emoji_pattern.sub(r'', str(data_json['reviewBody']))
            #remove emojis from title
            title_str = emoji_pattern.sub(r'', str(data_json['title']))

            if(canITranslate):

                #translatting
                review_str = googleTranslator(review_str, target)
                title_str = googleTranslator(title_str, target)
                #assingement
                data_json['title'] = title_str['translatedText']
                data_json['reviewBody'] = review_str['translatedText']

            if( i > 95000): #manual_fix
                #write
                data_enUS.write(json.dumps(data_json) + "\n")
                print('p4',i)

            if( data_json['reviewId'] == latestId):
                canITranslate = True
                print('I can translate now!!')

            if( data_json['reviewId'] != latestId and canITranslate == False):
                print('looking for')

        except KeyError:
            print("there is a key error")
            data_error.write(json.dumps(aux) + "\n")

    #close files streams
    data_enUS.close()
    dataset.close()
    data_error.close()
    print('data translate finish!!')

load()

looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
look

looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
look

looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
look

looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
look

looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
look

...

looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
look

looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
look

looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
look

looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
looking for
look

Forbidden: 403 POST https://translation.googleapis.com/language/translate/v2?prettyPrint=false: User Rate Limit Exceeded

<font size="4">**Calculate subjectivity and polarity from sentences**</font>

In [19]:
import statistics

data_us = open('data/dataset-full.txt', 'r', encoding='utf-8')
data_result = open('data/dataset-resultv2.txt', 'a', encoding='utf-8')

evaluations = {}
cities = {}
mobile =0   
        
for sentence in data_us:    
    try:
        
        data = sentence.replace('\\', '').replace('\n', '')
        data = json.loads(data)
        calc = TextBlob(data['reviewBody'])
        
        data['subjectivity'] = calc.sentiment.subjectivity
        data['polarity'] = calc.sentiment.polarity
        
        #how many mobile users ?
        if(str(data['mobileDevice']) == 'True'):
            mobile += 1
        
        if data['restaurantId'] in evaluations:
            
            evaluations[data['restaurantId']]['evaluations'].append(data['polarity'])
            evaluations[data['restaurantId']]['total']= evaluations[data['restaurantId']]['total'] + 1
            evaluations[data['restaurantId']]['stars'].append(int(data['ratingValue']))
            if( data['userLocation'] in evaluations[data['restaurantId']]['userLocations']):
                #do nothing
                print('')
            else:
                evaluations[data['restaurantId']]['userLocations'].append(data['userLocation'])

        else:
            evaluations[data['restaurantId']] = { 'evaluations': [], 'media': 0, 'userLocations':[], 'media_star': 0, 'total': 0,  'rest_name': '', 'stars': [] }
            evaluations[data['restaurantId']]['evaluations'].append(data['polarity'])
            evaluations[data['restaurantId']]['total'] = evaluations[data['restaurantId']]['total'] + 1
            evaluations[data['restaurantId']]['rest_name'] = data['itemReviewed']['name']
            evaluations[data['restaurantId']]['stars'].append(int(data['ratingValue']))
            evaluations[data['restaurantId']]['userLocations'].append(data['userLocation'])
                

        data_result.write(json.dumps(data) + "\n")

    except Exception as exc:
        print(exc)
    

data_us.close()
data_result.close()



























































































Expecting ',' delimiter: line 1 column 640 (char 639)


































































































































































































































































































































































































































































































































































































































































































































































































































































Expecting ',' delimiter




























































































































































































































































































































































Expecting ',' delimiter: line 1 column 13 (char 12)



























































































































































































































































































































































































































































































































































































































<font size="4">**Geral data**</font>

<font size="3">**Sentence Distribution**</font>

In [53]:
dataset = open('data/dataset-resultv2.txt', 'r', encoding='utf-8')
data_star = {}


def galplot(chart):
    rendered_chart = chart.render(is_unicode=True)
    plot_html = base_html.format(rendered_chart=rendered_chart)
    display(HTML(plot_html))
        
for sentence in dataset:
    try:
        data = json.loads(sentence)
        tam = len(data['reviewBody'].split())
        
        if ( data['ratingValue'] in data_star):
            data_star[data['ratingValue']]['words'].append(tam) 
        else:
            data_star[data['ratingValue']] = {'words': []}
            data_star[data['ratingValue']]['words'].append(tam) 
            
    except Exception as exc:
        print(exc)


one   = statistics.median(data_star['1']['words'])
two   = statistics.median(data_star['2']['words'])
three = statistics.median(data_star['3']['words'])
four  = statistics.median(data_star['4']['words'])
five  = statistics.median(data_star['5']['words'])

line_chart_pol = pygal.HorizontalBar()
line_chart_pol.title = 'Words Average by Stars'
line_chart_pol.x_labels = sorted(data_star.keys())

line_chart_pol.add('Words', [one, two, three, four, five])

galplot(line_chart_pol)


        
dataset.close()

<font size="3">**Comment by day week**</font>

In [28]:
dataset = open('data/dataset-resultv2.txt', 'r', encoding='utf-8')

def find_day(date):
    return datetime.datetime.strptime(date, '%d %m %Y').weekday()

def galplot(chart):
    rendered_chart = chart.render(is_unicode=True)
    plot_html = base_html.format(rendered_chart=rendered_chart)
    display(HTML(plot_html))
        
sunday = 0
sunday_pos = 0
sunday_neg = 0
sunday_neu = 0
monday = 0
monday_pos = 0
monday_neg = 0
monday_neu = 0
tuesday = 0
tuesday_pos = 0
tuesday_neg = 0
tuesday_neu = 0
wednesday = 0
wednesday_pos = 0
wednesday_neg = 0
wednesday_neu = 0
thursday = 0
thursday_pos = 0
thursday_neg = 0
thursday_neu = 0
friday = 0
friday_pos = 0
friday_neg = 0
friday_neu = 0
saturday = 0
saturday_pos = 0
saturday_neg = 0
saturday_neu = 0


for sentence in dataset:
    try:
        data = json.loads(sentence)
        date = data['publicationDate'].replace('/', ' ')
        day = find_day(date)

        if(day == 0):
            monday += 1
            if(data['polarity'] > 0):
                monday_pos += 1
            elif(data['polarity'] < 0):
                monday_neg += 1
            else:
                monday_neu += 1
        if(day == 1):
            sunday += 1
            if(data['polarity'] > 0):
                sunday_pos += 1
            elif(data['polarity'] < 0):
                sunday_neg += 1
            else:
                sunday_neu += 1
        if(day == 2):
            tuesday += 1
            if(data['polarity'] > 0):
                tuesday_pos += 1
            elif(data['polarity'] < 0):
                tuesday_neg += 1
            else:
                tuesday_neu += 1
        if(day == 3):
            wednesday += 1
            if(data['polarity'] > 0):
                wednesday_pos += 1
            elif(data['polarity'] < 0):
                wednesday_neg += 1
            else:
                wednesday_neu += 1
        if(day == 4):
            thursday += 1
            if(data['polarity'] > 0):
                thursday_pos += 1
            elif(data['polarity'] < 0):
                thursday_neg += 1
            else:
                thursday_neu += 1
        if(day == 5):
            friday += 1
            if(data['polarity'] > 0):
                friday_pos += 1
            elif(data['polarity'] < 0):
                friday_neg += 1
            else:
                friday_neu += 1
        if(day == 6):
            saturday += 1
            if(data['polarity'] > 0):
                saturday_pos += 1
            elif(data['polarity'] < 0):
                saturday_neg += 1
            else:
                saturday_neu += 1
    except Exception as exc:
        print(exc)

line_chart_pol = pygal.HorizontalBar()
line_chart_pol.title = 'Relation of comments by day'
line_chart_pol.x_labels = ['Sunday', 'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday']
line_chart_pol.add('Positive', [sunday_pos, monday_pos, tuesday_pos, wednesday_pos, thursday_pos,
                          friday_pos, saturday_pos])
line_chart_pol.add('Negative', [sunday_neg, monday_neg, tuesday_neg, wednesday_neg, thursday_neg,
                          friday_neg, saturday_neg])
line_chart_pol.add('Neutral', [sunday_neu, monday_neu, tuesday_neu, wednesday_neu, thursday_neu,
                          friday_neu, saturday_neu])
galplot(line_chart_pol)

<font size="3">**Most Evaluated**</font>

In [23]:
def galplot(chart):
    rendered_chart = chart.render(is_unicode=True)
    plot_html = base_html.format(rendered_chart=rendered_chart)
    display(HTML(plot_html))

arr = []

for i in evaluations:
    evaluations[i]['media'] = statistics.median(evaluations[i]['evaluations'])
    evaluations[i]['media_star'] = statistics.median(evaluations[i]['stars'])
    arr.append(evaluations[i])


dd = sorted(arr, key = lambda i: i['total'], reverse=True)[:10]
mm = sorted(arr, key = lambda i: i['media_star'], reverse=True)

line_total_rank_chart = pygal.HorizontalBar()
line_total_rank_chart.title = 'Most Revieweds'

for d in dd:
    line_total_rank_chart.add(d['rest_name'], d['total'])
    print(len(d['userLocations']))

galplot(line_total_rank_chart)


restaurant_rank_chart = pygal.HorizontalBar()
restaurant_rank_chart.title = 'Start Ranking'
one_star = True
two_star = True
three_star = True
four_star = True
five_star = True

for rest in mm:
    if(rest['media_star'] == 1 and one_star):
        restaurant_rank_chart.add(rest['rest_name'], rest['media_star'])
        one_star = False
    if(rest['media_star'] == 2 and two_star):
        restaurant_rank_chart.add(rest['rest_name'], rest['media_star'])
        two_star = False
    if(rest['media_star'] == 3 and three_star):
        restaurant_rank_chart.add(rest['rest_name'], rest['media_star'])
        three_star = False
    if(rest['media_star'] == 4 and four_star):
        restaurant_rank_chart.add(rest['rest_name'], rest['media_star'])
        four_star = False
    if(rest['media_star'] == 5 and five_star):
        restaurant_rank_chart.add(rest['rest_name'], rest['media_star'])
        five_star = False

galplot(restaurant_rank_chart)

252
306
213
216
172
207
138
85
67
117


Bubble Map

<font size="3">**Polarity and subjectivity proportion**</font>

In [9]:
import pygal
from IPython.display import SVG, HTML, display

data_result = open('data/dataset-resultv2.txt', 'r', encoding='utf-8')

def galplot(chart):
    rendered_chart = chart.render(is_unicode=True)
    plot_html = base_html.format(rendered_chart=rendered_chart)
    display(HTML(plot_html))
        
pol_positive = 0
pol_negative = 0
pol_neutral = 0

sub_objective = 0
sub_subjective = 0

for result in data_result:
    data = json.loads(result)
    if(data['polarity'] > 0):
        pol_positive+=1
    elif(data['polarity'] < 0):
        pol_negative+=1
    else:
        pol_neutral+=1
    
    if(data['subjectivity'] < 0.5):
        sub_objective+=1
    else:
        sub_subjective+=1
    
pie_pol_chart = pygal.Pie(inner_radius=.4)
pie_pol_chart.title = 'Polarity Proportion'
pie_pol_chart.add('Positive', pol_positive)
pie_pol_chart.add('Negative', pol_negative)
pie_pol_chart.add('Neutral', pol_neutral)
galplot(pie_pol_chart)

pie_sub_chart = pygal.Pie(inner_radius=.4)
pie_sub_chart.title = 'Subjectivity Proportion'
pie_sub_chart.add('Objective', sub_objective)
pie_sub_chart.add('Subjective', sub_subjective)
galplot(pie_sub_chart)


pie_mobile_chart = pygal.Pie(inner_radius=.4)
pie_mobile_chart.title = 'Mobile Interactions'
pie_mobile_chart.add('Mobile', mobile)
pie_mobile_chart.add('Non Mobile', mobile - len(evaluations) ) # refactor using abs
galplot(pie_mobile_chart)


<font size="4">**Method Evaluation**</font>

<font size="3">**Creates a random dataset**</font>

In [33]:
from random import randint
import json
import csv

dataset = open('data/dataset-result.txt', 'r', encoding='utf-8')
size_dataset = 0

# Reading from file
content = dataset.read()
list_lines = content.split("\n")

# Count lines
for i in list_lines:
    if i:
        size_dataset += 1

index_memo = []

def get_especific_line(index, dataset):
    dataset = open('data/dataset-result.txt', 'r', encoding='utf-8')
    for i, sentence in enumerate(dataset):
        if i == index:
            return json.loads(sentence)

def get_random_value():
    index = randint(0, size_dataset)
    
    if(index in index_memo):
        while index in index_memo:
            index = randint(0, size_dataset)

    return index

with open('data/dataset-random-200-new.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["comment", "subjectivity", "polarity", "sub-check", "pol-check"])

    for x in range(0, 199):
        index = get_random_value()    
        line = get_especific_line(index, dataset)
        print(line)
        writer.writerow([line['reviewBody'], line['subjectivity'], line['polarity']])

{'title': 'Um dos melhores lanches de Su00e3o Paulo', 'reviewId': '564769750', 'restaurantId': '5704077', 'author': 'Kuantun', 'reviewBody': "Delicious hamburgers, note 10 service and fast. Excellent experience.In my opinion and that of friends who frequent it, Sampa's best snack bar / hamburger shop.", 'language': 'pt', 'mobileDevice': 'True', 'userLocation': 'Su00e3o Paulo, SP', 'publicationDate': '05/03/2018', 'visitDate': '02/2018', 'ratingValue': '5', 'itemReviewed': {'name': 'Davis', 'address': {'streetAddress': 'Rua Padre Guido del Toro 519 Ch Inglesa', 'addressLocality': 'Su00e3o Paulo', 'addressRegion': 'Estado de Su00e3o Paulo', 'postalCode': '05142-120', 'country': 'Brasil'}, 'website': 'http://www.davislanchonete.com.br', 'email': 'None', 'phoneNumber': '1136445803', 'averageRating': '4.5'}, 'subjectivity': 1, 'polarity': 1}
{'title': 'u00d3tima comida pra ir sem pressa!!', 'reviewId': '562802832', 'restaurantId': '10644044', 'author': '161eduardog', 'reviewBody': 'Beautifu

{'title': 'Avaliau00e7u00e3o', 'reviewId': '244141567', 'restaurantId': '5351147', 'author': 'Carlos N', 'reviewBody': 'Good place to eat a quick barbecue, with a good price, cold beer, several options on the menu, but ask for the service, very slow waiters.', 'language': 'pt', 'mobileDevice': 'False', 'userLocation': 'None', 'publicationDate': '12/12/2014', 'visitDate': '12/2014', 'ratingValue': '3', 'itemReviewed': {'name': 'Estilo Gaucho', 'address': {'streetAddress': 'Praca Dodo E Osmar 8 Centro', 'addressLocality': 'Salvador', 'addressRegion': 'Bahia', 'postalCode': '40050-066', 'country': 'Brasil'}, 'website': 'None', 'email': 'None', 'phoneNumber': 'None', 'averageRating': '4.0'}, 'subjectivity': 1, 'polarity': 1}
{'title': 'Razou00e1vel', 'reviewId': '431995868', 'restaurantId': '2320355', 'author': 'TaliMonteRosa', 'reviewBody': 'The place is beautiful and attracts a little patricinha / mauricinho crowd. It is by the sea at Boa Viagem Beach. It is a mix of pizzeria, with typic

<font size="3">**Open classified data**</font>

In [24]:
import pandas as pd

dataset = pd.read_csv("data/dataset-200-classified.csv", skiprows = 2)

vp_sub = 0
fp_sub = 0
fn_sub = 0

for line, rows in dataset.iterrows():
    #VP
    if(rows['sub-check'] == 1 and rows['subjectivity'] == 1):
        vp_sub += 1
    #FP
    if(rows['sub-check'] == 0 and rows['subjectivity'] == 1):
        fp_sub +=1
    #FN
    if(rows['sub-check'] == 1 and rows['subjectivity'] == 0):
        fn_sub +=1

<font size="3">**Precision subjectivity**</font>

In [25]:
precision = vp_sub/(vp_sub+fp_sub)
print(precision * 100)

92.10526315789474


<font size="3">**Recall**</font>

In [26]:
recall = vp_sub/(vp_sub+fn_sub)
print(recall * 100)

79.54545454545455


<font size="3">**F1-measure**</font>

In [27]:
f1 = (2*(precision*recall))/(precision+recall)
print(f1 * 100)

85.36585365853658
